In [2]:
#Importing the git repo, for code reuse
!git clone https://github.com/daniel-kolo/you_are_fake_news.git

Cloning into 'you_are_fake_news'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 174 (delta 23), reused 39 (delta 11), pack-reused 120
Receiving objects: 100% (174/174), 142.23 MiB | 23.47 MiB/s, done.
Resolving deltas: 100% (51/51), done.
Checking out files: 100% (24/24), done.


In [3]:
# Mounting the google drive, where we've kept the downloaded embeddings, and the unsharded data
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Opening and loading the data
with open("../../gdrive/My Drive/reliable_news_prep", "r") as reliable_file:
    rel = [line.strip() for line in reliable_file]
with open("../../gdrive/My Drive/fake_news_prep", "r") as fake_file:
    fake = [line.strip() for line in fake_file]
    

In [0]:
#Cleaning the data from the headline-s, it could be the source of the suprinsingly high accuracy values
clean_fake = []
for line in fake:
    if ("blockchain" in line) and ("bitcoin" in line) and ("headline" in line):
        line = line.split()
        line = line[10:]
        line = (" ").join(line)
    clean_fake.append(line)
    
clean_fake[0]
fake = clean_fake

In [9]:
# Setting the training length
length=50000

text = rel[:length]+fake[:length]
labels = [0 if i<length else 1 for i in range(2*length)] # reliable - 0; fake - 1

from training_preprocess import prepare_fastText_embedding_matrix as prepare_embedding
from training_preprocess import sequence_vectorize
from training_preprocess import train_val_test_split as split

# Train, Test, Valid dataset creation
X_train, X_val, X_test, Y_train, Y_test, Y_val = split(text, labels, 0.2, 0.1)
x_train, x_val, x_test, word_index = sequence_vectorize(X_train, X_val, X_test, sequence_length=1100)

#preparing the embedding matrix
embedding_matrix = prepare_fastText_embedding_matrix(word_index, "../../gdrive/My Drive/cc.en.300.bin")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [0]:
from sklearn.model_selection import train_test_split
from gensim.models import FastText, fasttext
from tensorflow.python.keras.preprocessing import sequence
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# Had to redefine the function, because Google colab couldn't use gensim.models
# Couldn't figure out why, had to use a different vector loading method

def prepare_fastText_embedding_matrix(word_index, model_file):
    #model = gensim.models.fasttext.load_facebook_model(model_file)
    model=FastText.load_fasttext_format(model_file)
    EMBEDDING_DIM = model.vector_size
    embedding_matrix = np.zeros((len(word_index)+1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = model[word]
        embedding_matrix[i-1] = embedding_vector
    
    return embedding_matrix

In [10]:
from keras import layers, models, optimizers, regularizers


Using TensorFlow backend.


In [0]:
import numpy as np

# Vectorizing the Fake News - Reliable News labels
def label_to_vector(labels):
    array = np.zeros([len(labels), 2])
    for i, label in enumerate(labels):
        array[i, label] = 1
    return array

In [0]:

#Defining neural net
def bidirectional_LSTM(embedding_dim, dropout_rate, input_shape, num_classes, is_embedding_trainable, embedding_matrix):
    
    # Add an Input Layer
    input_layer = layers.Input((input_shape, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(input_dim=embedding_matrix.shape[0],
                                       output_dim=embedding_dim,
                                       weights=[embedding_matrix],
                                       trainable=is_embedding_trainable)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.6)(embedding_layer)
    
    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.LSTM(20))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(70, activation="relu")(lstm_layer)
    output_layer2 = layers.Dropout(dropout_rate)(output_layer1)
    output_layer3 = layers.Dense(num_classes, activation="softmax")(output_layer2)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer3)
    
    return model

In [0]:
y_train = label_to_vector(Y_train)

In [0]:
y_val = label_to_vector(Y_val)


In [0]:
# First suspiciously high accuracy rates
import keras

model = bidirectional_LSTM(300, 0.2, 1100, 2, True, embedding_matrix)
model.compile(optimizer='Adam', loss="binary_crossentropy", metrics=['accuracy'])

callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]

model.fit(x_train,
          y_train,
          epochs=10,
          callbacks=callbacks,
          validation_data=(x_val, y_val),
          verbose=2,  # Logs once per epoch.
          batch_size=128)

model.save_weights('../../gdrive/My Drive/sequence_model_with_pre_trained_embedding.h5')

Train on 70000 samples, validate on 20000 samples
Epoch 1/10
 - 2874s - loss: 0.2253 - acc: 0.9083 - val_loss: 0.1916 - val_acc: 0.9223
Epoch 2/10
 - 2875s - loss: 0.1335 - acc: 0.9508 - val_loss: 0.1415 - val_acc: 0.9498
Epoch 3/10
 - 2849s - loss: 0.1127 - acc: 0.9582 - val_loss: 0.1439 - val_acc: 0.9513
Epoch 4/10
 - 2854s - loss: 0.1004 - acc: 0.9629 - val_loss: 0.1351 - val_acc: 0.9525
Epoch 5/10
 - 2838s - loss: 0.0804 - acc: 0.9702 - val_loss: 0.1470 - val_acc: 0.9487
Epoch 6/10
 - 2923s - loss: 0.0613 - acc: 0.9784 - val_loss: 0.1279 - val_acc: 0.9617
Epoch 7/10
 - 2938s - loss: 0.0470 - acc: 0.9835 - val_loss: 0.1265 - val_acc: 0.9634
Epoch 8/10
 - 2943s - loss: 0.0428 - acc: 0.9850 - val_loss: 0.1447 - val_acc: 0.9616
Epoch 9/10
 - 2894s - loss: 0.0365 - acc: 0.9872 - val_loss: 0.1484 - val_acc: 0.9660
Epoch 10/10
 - 2854s - loss: 0.0260 - acc: 0.9908 - val_loss: 0.1620 - val_acc: 0.9650


#Parameter optimalization

In [0]:
!git clone https://github.com/daniel-kolo/you_are_fake_news.git

Cloning into 'you_are_fake_news'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 171 (delta 21), reused 38 (delta 11), pack-reused 120
Receiving objects: 100% (171/171), 142.23 MiB | 20.43 MiB/s, done.
Resolving deltas: 100% (49/49), done.
Checking out files: 100% (25/25), done.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
with open("../../gdrive/My Drive/reliable_news_prep", "r") as reliable_file:
    rel = [line.strip() for line in reliable_file]
with open("../../gdrive/My Drive/fake_news_prep", "r") as fake_file:
    fake = [line.strip() for line in fake_file]

In [0]:
clean_fake = []
for line in fake:
    if ("blockchain" in line) and ("bitcoin" in line) and ("headline" in line):
        line = line.split()
        line = line[10:]
        line = (" ").join(line)
    clean_fake.append(line)
    
fake = clean_fake

In [46]:
# The actual parameter optimalization
dropout_rates = [0, 0.5]
optimizers= ['rmsprop', 'adam']

y_train = label_to_vector(Y_train)
y_val = label_to_vector(Y_val)

import keras
counter = 0


for rate in dropout_rates:
  for optimizer in optimizers:
    
    print('a modell hiperparaméterei: ')
    #print('dimension: {}'.format(dimension))
    print('dropout rate: {}'.format(rate))
    print('optimizer: {}'.format(optimizer))

    model = bidirectional_LSTM(300, rate, 1100, 2, True, embedding_matrix)
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=['accuracy'])
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]

    result = model.fit(x_train,
              y_train,
              epochs=10,
              callbacks=callbacks,
              validation_data=(x_val, y_val),
              verbose=2,  # Logs once per epoch.
              batch_size=128)

    model.save_weights('../../gdrive/My Drive/sequence_model_with_pre_trained_embedding_{}.h5'.format(counter))
    counter +=1



a modell hiperparaméterei: 
dropout rate: 0
optimizer: rmsprop
Train on 27999 samples, validate on 8001 samples
Epoch 1/10
 - 985s - loss: 0.2142 - acc: 0.9180 - val_loss: 0.1198 - val_acc: 0.9616
Epoch 2/10
 - 979s - loss: 0.1115 - acc: 0.9610 - val_loss: 0.0983 - val_acc: 0.9695
Epoch 3/10
 - 985s - loss: 0.0884 - acc: 0.9701 - val_loss: 0.0914 - val_acc: 0.9726
Epoch 4/10
 - 979s - loss: 0.0762 - acc: 0.9743 - val_loss: 0.1044 - val_acc: 0.9689
Epoch 5/10
 - 977s - loss: 0.0674 - acc: 0.9771 - val_loss: 0.0958 - val_acc: 0.9715
Epoch 6/10
 - 967s - loss: 0.0574 - acc: 0.9803 - val_loss: 0.1063 - val_acc: 0.9634
a modell hiperparaméterei: 
dropout rate: 0
optimizer: adam
Train on 27999 samples, validate on 8001 samples
Epoch 1/10
 - 974s - loss: 0.2522 - acc: 0.8900 - val_loss: 0.1306 - val_acc: 0.9543
Epoch 2/10
 - 968s - loss: 0.1020 - acc: 0.9639 - val_loss: 0.1436 - val_acc: 0.9530
Epoch 3/10
 - 952s - loss: 0.0623 - acc: 0.9790 - val_loss: 0.1192 - val_acc: 0.9605
Epoch 4/10
 - 

# Final model training

In [16]:
# Last training cycle after parameter optimalization, with all the data
import keras

model = bidirectional_LSTM(300, 0.5, 1100, 2, True, embedding_matrix)
model.compile(optimizer='rmsprop', loss="binary_crossentropy", metrics=['accuracy'])

callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]

model.fit(x_train,
          y_train,
          epochs=10,
          callbacks=callbacks,
          validation_data=(x_val, y_val),
          verbose=2,  # Logs once per epoch.
          batch_size=128)

model.save_weights('../../gdrive/My Drive/final_sequence_model_with_pre_trained_embedding.h5')

model.summary()


Train on 70000 samples, validate on 20000 samples
Epoch 1/10
 - 2565s - loss: 0.2222 - acc: 0.9108 - val_loss: 0.1396 - val_acc: 0.9487
Epoch 2/10
 - 2603s - loss: 0.1369 - acc: 0.9499 - val_loss: 0.1440 - val_acc: 0.9497
Epoch 3/10
 - 2617s - loss: 0.1145 - acc: 0.9576 - val_loss: 0.1170 - val_acc: 0.9570
Epoch 4/10
 - 2597s - loss: 0.1039 - acc: 0.9625 - val_loss: 0.1040 - val_acc: 0.9603
Epoch 5/10
 - 2543s - loss: 0.0932 - acc: 0.9662 - val_loss: 0.1030 - val_acc: 0.9603
Epoch 6/10
 - 2560s - loss: 0.0853 - acc: 0.9701 - val_loss: 0.0964 - val_acc: 0.9637
Epoch 7/10
 - 2650s - loss: 0.0791 - acc: 0.9715 - val_loss: 0.0998 - val_acc: 0.9683
Epoch 8/10
 - 2707s - loss: 0.0708 - acc: 0.9749 - val_loss: 0.0971 - val_acc: 0.9686
Epoch 9/10
 - 2647s - loss: 0.0630 - acc: 0.9787 - val_loss: 0.0819 - val_acc: 0.9719
Epoch 10/10
 - 2649s - loss: 0.0571 - acc: 0.9802 - val_loss: 0.0784 - val_acc: 0.9722
Model: "model_2"
_________________________________________________________________
Layer 